In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

Question 1 
------------------------------------------------------------------------------

In [27]:
data = pd.read_csv("data/25_Portfolios_5x5.csv",skiprows=15,index_col="DATE",parse_dates=True,nrows=1134)
data = data.dropna()
data.index = pd.to_datetime(data.index,format="%Y%m")
factors = pd.read_csv("data/F-F_Research_Data_Factors.csv",skiprows=3,index_col=0,parse_dates=True,nrows=1134)
factors = factors.dropna()
factors.index = pd.to_datetime(factors.index,format="%Y%m")
rm = factors["Mkt-RF"]
rf = factors["RF"]

In [28]:
data = data["1963-07-01":"2020-06-01"]
rm = rm["1963-07-01":"2020-06-01"]
rf = rf["1963-07-01":"2020-06-01"]

In [40]:
arith_mean = data.mean(0)
simple_mean = pd.DataFrame(np.zeros((5,5)),index=["SMALL","ME2","ME3","ME4","BIG"],columns=["LoBM","BM2","BM3","BM4","HiBM"])
for r in range(0,5):
    for c in range(0,5):
        n = r * 5 + c
        simple_mean.iloc[r,c] = arith_mean[n]
simple_mean.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.67,1.17,1.13,1.31,1.39
ME2,0.89,1.13,1.22,1.22,1.30
ME3,0.92,1.16,1.09,1.23,1.29
ME4,1.02,0.99,1.05,1.17,1.17
BIG,0.92,0.89,0.92,0.85,0.98


In [30]:
excess_data = data.subtract(rf,axis=0)

In [31]:
betas = {}
for column in excess_data:
    res = sm.OLS(excess_data[column],rm,cov_type="HC0").fit()
    betas[column] = res.params
betas = pd.DataFrame(betas).T

betas = betas.squeeze()

In [39]:
beta_table = pd.DataFrame(np.zeros((5,5)),index=["SMALL","ME2","ME3","ME4","BIG"],columns=["LoBM","BM2","BM3","BM4","HiBM"])
for r in range(0,5):
    for c in range(0,5):
        n = r * 5 + c
        beta_table.iloc[r,c] = betas[n]
beta_table.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,1.41,1.25,1.12,1.05,1.09
ME2,1.39,1.19,1.07,1.03,1.16
ME3,1.32,1.13,1.02,1.00,1.09
ME4,1.22,1.09,1.02,0.99,1.10
BIG,0.98,0.93,0.87,0.89,0.98


In [37]:
# Standard deviation
stds = data.std(0)
std_dev = pd.DataFrame(np.zeros((5,5)),index=["SMALL","ME2","ME3","ME4","BIG"],columns=["LoBM","BM2","BM3","BM4","HiBM"])
for r in range(0,5):
    for c in range(0,5):
        n = r * 5 + c
        std_dev.iloc[r,c] = stds[n]
#print(simple_mean.to_latex())
std_dev.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,7.92,6.95,6.02,5.78,6.06
ME2,7.16,6.02,5.45,5.33,6.18
ME3,6.57,5.49,5.01,5.05,5.79
ME4,5.87,5.14,4.99,4.97,5.76
BIG,4.60,4.41,4.31,4.72,5.57


In [34]:
# Sharpe ratio
sharpe = ((simple_mean - rf.mean())/std_dev) * np.sqrt(12)
sharpe

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.13,0.40,0.43,0.56,0.58
ME2,0.25,0.43,0.54,0.55,0.52
ME3,0.29,0.49,0.49,0.59,0.55
ME4,0.38,0.41,0.47,0.55,0.48
BIG,0.41,0.40,0.44,0.35,0.38


In [42]:
nobs = data.shape[0]
tstats = (simple_mean / (std_dev/np.sqrt(nobs)) )
tstats.round(2)

,LoBM,BM2,BM3,BM4,HiBM
SMALL,2.22,4.39,4.92,5.92,6.01
ME2,3.25,4.93,5.84,6.00,5.51
ME3,3.65,5.54,5.70,6.39,5.80
ME4,4.56,5.04,5.50,6.16,5.31
BIG,5.26,5.31,5.60,4.72,4.62


In [51]:
data

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
DATE,,,,,,,,,,,,,,,,,,,,,
1963-07-01,1.1307,-0.1481,0.5563,0.1097,-1.3212,-1.8071,0.1899,-0.8445,-1.9644,-1.1889,...,-0.8456,-1.7052,-2.0042,-1.6657,-1.8567,0.1401,0.4524,1.2557,-0.4290,-1.1026
1963-08-01,4.2370,1.4831,1.3214,2.3701,4.6745,5.5665,4.5191,4.3639,4.4188,8.2329,...,5.3965,4.7614,6.2068,7.5619,5.3469,5.7710,4.2230,4.7487,8.1419,6.3824
1963-09-01,-2.8878,0.7027,-1.0869,-1.5812,-0.3581,-4.0502,-1.5034,-0.6417,-1.1812,-2.9149,...,-2.6893,-2.0139,-1.8941,-3.5823,-1.9943,-1.3721,-0.7720,-0.9852,-0.1905,-3.4963
1963-10-01,1.2885,-0.6491,1.2443,0.1000,2.3724,1.1916,4.2342,2.3438,2.2012,3.9335,...,-0.4287,0.9057,2.3459,5.3382,0.6113,5.3261,1.7383,-0.2829,2.4150,0.4857
1963-11-01,-3.3751,-4.0355,-1.5969,-1.0548,-1.0454,-4.2561,-1.7534,-0.7064,-0.1002,-0.1132,...,-0.8648,-0.8828,-0.4882,1.1451,3.5388,-1.2561,0.9849,-1.5542,-2.1322,1.3455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-01,-6.1676,-4.2818,-4.1692,-8.4173,-8.5877,-7.5118,-8.3567,-10.8356,-10.2463,-10.5911,...,-6.9096,-7.9617,-8.8968,-10.8475,-13.4285,-6.9742,-7.9885,-8.9713,-10.7681,-11.7101
2020-03-01,-21.3965,-14.9664,-21.9287,-26.7859,-27.8195,-19.8626,-19.5344,-21.2570,-23.7718,-32.1435,...,-12.3195,-18.6616,-21.0827,-31.4269,-32.8024,-7.7416,-11.2389,-15.5893,-20.1142,-28.4009
2020-04-01,24.3797,20.8938,14.5475,11.4337,15.8626,19.2871,12.8404,11.9620,9.8927,26.0187,...,16.7807,14.0032,11.8580,16.2333,17.7799,14.0851,13.6907,9.7413,10.4612,15.5348


In [96]:
long_short_row = {}
for i in range(4,25,5):
    long_short_row[i] = data.iloc[:,i] - data.iloc[:,i-4]
long_short_row = pd.DataFrame(long_short_row)
lsr_tstat = (long_short_row.mean() / (long_short_row.std() / np.sqrt(nobs))) 

lsr_tstat = pd.DataFrame(lsr_tstat,columns=["Long Short"])
lsr_tstat.index = ["SMALL","ME2","ME3","ME4","BIG"]

tstats["Long Short"] = lsr_tstat
tstats

,LoBM,BM2,BM3,BM4,HiBM,Long Short
SMALL,2.215130,4.386773,4.921799,5.920691,6.006604,4.573738
ME2,3.245961,4.925551,5.842019,6.003727,5.505203,2.686372
ME3,3.652665,5.537041,5.704868,6.391216,5.803893,2.244093
ME4,4.563582,5.039480,5.500083,6.157275,5.313279,0.914495
BIG,5.256925,5.310569,5.600534,4.718423,4.615675,0.358394


In [94]:
long_short_col = {}
for i in range(20,25):
    long_short_col[i] = data.iloc[:,i] - data.iloc[:,i-20]

long_short_col = pd.DataFrame(long_short_col)
lsc_tstat = (long_short_col.mean() / (long_short_col.std() / np.sqrt(nobs))) 

lsc_tstat = pd.DataFrame(lsc_tstat,columns=["Long Short"]).T
lsc_tstat.columns = ["LoBM","BM2","BM3","BM4","HiBM"]

#tstats["Long Short"] = lsc_tstat
tstats_extended = tstats.append(lsc_tstat)
tstats_extended.round(2)

,LoBM,BM2,BM3,BM4,HiBM,Long Short
SMALL,2.22,4.39,4.92,5.92,6.01,4.57
ME2,3.25,4.93,5.84,6.00,5.51,2.69
ME3,3.65,5.54,5.70,6.39,5.80,2.24
ME4,4.56,5.04,5.50,6.16,5.31,0.91
BIG,5.26,5.31,5.60,4.72,4.62,0.36
Long Short,1.16,-1.36,-1.22,-2.70,-2.26,NaN
